### HyperDrive

In [1]:
# setting and install properly requirements for AutoML
!pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

In [2]:
#!pip install zipp==3.3.1

In [3]:
#!pip install cryptography==3.1.1

In [2]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'b19c9c81-5f59-4537-b3c0-c1beb163ec22'
resource_group = 'aml-quickstarts-126392'
workspace_name = 'quick-starts-ws-126392'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Bank-marketing')
dataset.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [3]:
from azureml.core import Workspace, Experiment
# wcześniej trzeba uruchomić w AzureML Studio przestrzeń roboczą o nazwie "udacity-project"
ws = workspace
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
#uruchamiamy logowanie
run = exp.start_logging()

Workspace name: quick-starts-ws-126392
Azure region: southcentralus
Subscription id: b19c9c81-5f59-4537-b3c0-c1beb163ec22
Resource group: aml-quickstarts-126392


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-cluster"
# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "udacity-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())



found compute target: udacity-cluster


In [9]:
%%writefile train.py

from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
# import logging

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
url_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

run = Run.get_context()

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df, y_df
    

    
    
ds = TabularDatasetFactory.from_delimited_files(url_data)
x, y = clean_data(ds)

# TODO: Split data into train and test sets.
x_train, x_test, y_train, y_test = train_test_split(x, y)

    
    
def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

    args = parser.parse_args()

    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))

    model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

    accuracy = model.score(x_test, y_test)
    run.log("Accuracy", np.float(accuracy))

if __name__ == '__main__':
    main()


Writing train.py


In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os
# Specify parameter sampler
#ps = RandomParameterSampling( {
#    "--C": choice([0.00001,0.001,0.1,1,10,50,100,10000]),
#    "--max_iter": choice(range(10, 200))
#   }
#)
# Specify parameter sampler
params = {
	"--C": uniform(0.5, 1.0),
	"--max_iter": choice(100, 150, 200)
}
print("setting parameter sampler")
ps =  RandomParameterSampling(params)
print("specify random parameter sampling")

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, 
                      evaluation_interval=1, 
                      delay_evaluation=5) 
print("Specify a Policy")
# trzeba będzie sprawdzić gdzie zapisuje się efekt eksperymentu ćzy nie w folderze training
if "training" not in os.listdir():
    print("Creating new directory training")
    os.mkdir("./training")
else:
    print("Directory training existed")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",
              compute_target=compute_target,
              entry_script="train.py")
print("created SKLearn estimator")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
	estimator=est,
	hyperparameter_sampling=ps,
	policy=policy,
	primary_metric_name="Accuracy",
	primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
	max_concurrent_runs=4,
	max_total_runs=12)   
# można ustawić sobie max_total_runs=25

print("created hyperdrive_config")

setting parameter sampler
specify random parameter sampling
Specify a Policy
Directory training existed
created SKLearn estimator
created hyperdrive_config


In [24]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
# wysyłamy ustawienia hyperparametrów do eksperymentu żeby pokaząc uruchomienie
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [25]:
import joblib
# Get your best run and save the model from that run.
# coś wydaje mi się że jest pobranie wyników ale nie ma ustawień 
best_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=best_hyperdrive.get_details()['runDefinition']['arguments'], filename=os.path.join('training', 'best_hyperdrive_model.joblib'))
# nie  bardzo rozumiem tego kodu
best_hyperdrive.download_files("./training/best_model_hyper.joblib", "logreg_best_model.joblib")




In [28]:
# zarejstrowanie modelu jako "bankmodel"
model = best_hyperdrive.register_model(model_name='bank_hyper_model', model_path='azureml-logs/model.joblib')

In [29]:
# ustawiamy parametry i sprawdzamy najlepszy wynik czyli Accuracy
best_run_metrics = best_hyperdrive.get_metrics()
parameter_values = best_hyperdrive.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_hyperdrive.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', parameter_values[1])
print('\n max_iter:', parameter_values[3])

Best Run Id:  HD_76435390-9a63-4fb0-9981-4b6a386e39eb_5

 Accuracy: 0.9118718135469774

 C: 0.8626611368448052

 max_iter: 150


### Auto ML

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(url_data)


In [6]:
# ten kod trzeba mocno zmienić
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y)
train_data = pd.concat([x_train,y_train],axis = 1)

In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    experiment_exit_score = 0.96,
    enable_onnx_compatible_models=True,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= train_data,
    label_column_name= 'y',
    n_cross_validations= 5,
    enable_early_stopping = True,
    iteration_timeout_minutes = 5,
    featurization = 'auto')

In [8]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)


Running on local machine
Parent Run ID: AutoML_469ddf0d-363b-4851-a165-f82f19f3b030

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [12]:
# do uruchomienia wizualnego wyników metryk
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
print(automl_run)

Run(Experiment: udacity-project,
Id: AutoML_469ddf0d-363b-4851-a165-f82f19f3b030_30,
Type: None,
Status: Completed)


In [32]:
#automl_run, best_automl_model = automl_run.get_output()
print(best_automl_model)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
 # Retrieve and save your best automl model.
# automl_run, best_automl_model = automl_run.get_output(return_onnx_model=True)
# print(best_automl_model)? podejrzewam onnx model i zrobić to bez onnx
import joblib
joblib.dump(value=best_automl_model, filename='./training/best_automl_model.joblib')
# rejestracja modelu 
# register_model_automl = automl_run.register_model(model_name="bank_automl_model", model_path="best_automl_model.joblib")
# register_model_automl = automl_run.register_model(model_name="bank_automl_model")


['./training/best_automl_model.joblib']

In [21]:
# pobranie matryk i parametrów
best_run_metrics = automl_run.get_metrics()
parameter_values = automl_run.get_details()

print("Accuracy: ", best_run_metrics["accuracy"])
print("\n")
print("Parameters: ", parameter_values)
print("\n")
print("Metrics for best run: ", automl_run.get_metrics())

Accuracy:  0.9168422566837012


Parameters:  {'runId': 'AutoML_469ddf0d-363b-4851-a165-f82f19f3b030_30', 'status': 'Completed', 'startTimeUtc': '2020-11-16T00:04:05.42109Z', 'endTimeUtc': '2020-11-16T00:05:09.246729Z', 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'b19c9c81-5f59-4537-b3c0-c1beb163ec22\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_469ddf0d-363b-4851-a165-f82f19f3b030_30","experiment_name":null,"workspace_name":"quick-starts-ws-126392","subscription_id":"b19c9c81-5f59-4537-b3c0-c1beb163

In [22]:
# save into ONNX
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_file_path = "./training/best_automl_model.onnx"
OnnxConverter.save_onnx_model(best_automl_model, onnx_file_path)

In [35]:
# Delete Compute cluster
compute_target.delete()
#Delete the Compute Cluster
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster Deleted')
except:
    print('Unable to delete the cluster')

Cluster Deleted
